<h1 style="text-align:center">Regression Analysis for the California Housing census</h1>

<p style="text-align:center">Author: Jose Pena</p>
<p style="text-align:center">Github: <a href="https://github.com/JoseJuan98">JoseJuan98</a></p>
<br>

------------------------------------------------------------------------------------------------

<br>

## Aims

The aim of this notebook is to demonstrate the data regression skills leveraging a wide variety of tools, but also this report focuses on present findings, insights, and next steps.

-----------------------------------

## Instructions:

This exercise will demonstrate the data regression skills. You are expected to leverage a wide variety of tools, but also this report should focus on present findings, insights, and next steps. You may include some visuals from your code output, but this report is intended as a summary of your findings, not as a code review.

The development will center around 5 main points:

1.  Does the report include a section describing the data?
2.  Does the report include a paragraph detailing the main objective(s) of this analysis?
3.  Does the report include a section with variations of linear regression models and specifies which one is the model that best suits the main objective(s) of this analysis.
4.  Does the report include a clear and well-presented section with key findings related to the main objective(s) of the analysis?
5.  Does the report highlight possible flaws in the model and a plan of action to revisit this analysis with additional data or different predictive modeling techniques?


Once you have selected a data set, you will produce the deliverables listed below and submit them to one of your peers for review. Treat this exercise as an opportunity to produce analysis that are ready to highlight your analytical skills for a senior audience, for example, the Chief Data Officer, or the Head of Analytics at your company.
Sections required in your report:

*   Main objective of the analysis that specifies whether your model will be focused on prediction or interpretation.
*   Brief description of the data set you chose and a summary of its attributes.
*   Brief summary of data exploration and actions taken for data cleaning and feature engineering.
*   Summary of training at least three linear regression models which should be variations that cover using a simple  linear regression as a baseline, adding polynomial effects, and using a regularization regression. Preferably, all use the same training and test splits, or the same cross-validation method.
*   A paragraph explaining which of your regressions you recommend as a final model that best fits your needs in terms of accuracy and explainability.
*   Summary Key Findings and Insights, which walks your reader through the main drivers of your model and insights from your data derived from your linear regression model.
*   Suggestions for next steps in analyzing this data, which may include suggesting revisiting this model adding specific data features to achieve a better explanation or a better prediction.


-------------------------------------------

# Setup

For this Regression Data Analysis project the following libraries will be used:

*   [`pandas`](https://pandas.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0232ENSkillsNetwork30654641-2021-01-01) for managing the data.
*   [`numpy`](https://numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0232ENSkillsNetwork30654641-2021-01-01) for mathematical operations.
*   [`seaborn`](https://seaborn.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0232ENSkillsNetwork30654641-2021-01-01) for visualizing the data.
*   [`matplotlib`](https://matplotlib.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0232ENSkillsNetwork30654641-2021-01-01) for visualizing the data.
*   [`sklearn`](https://scikit-learn.org/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0232ENSkillsNetwork30654641-2021-01-01) for machine learning related functions.

## Imports

In [1]:
import pandas
import numpy
import seaborn

from scipy.stats import boxcox, ttest_ind
from matplotlib import pyplot
from pandas.plotting import scatter_matrix

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

# utils.py
from src.utils import fetch_housing_data, HOUSING_URL, HOUSING_PATH, plot_scatters, \
    plot_univariate_boxplots, remove_outliers_iqr, prepare_data, TARGET

Setting up some options:

In [2]:
# Display and store plot within the notebook
%matplotlib inline

# Show all columns when displaying dataframe
pandas.options.display.max_columns = None

 Load data, if it wasn't downloaded before it will fetch it from the URL specified, otherwise will load it from a local '.csv' file.

In [3]:
data = fetch_housing_data(url=HOUSING_URL,
                          path=HOUSING_PATH)

# 1. About the Data

This California Housing dataset is available from [Luís Torgo's page](https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html) (University of Porto).

This dataset appeared in a 1997 paper titled Sparse Spatial Autoregressions by Pace, R. Kelley and Ronald Barry, published in the Statistics and Probability Letters journal. They built it using the 1990 California census data. It contains one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).

The target variable or dependent variable for this anlysis will be the `median_house_value`, which describes median price of the houses per block group.

**Shape of the dataset**

In [4]:
data.shape

(20640, 10)

**List of columns**

In [5]:
data.columns.to_list()

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity']

**First 5 rows of the dataset**

In [6]:
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


**Non-null values count, type of feature and memory usage**

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


**Statistical properties of the dataset**

In [8]:
data.describe(include='all')

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<1H OCEAN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9136
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909,NaN
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874,NaN
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000,NaN
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000,NaN
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000,NaN
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000,NaN


As it was analyzed in the previous exercise of Exploratory Data Analysis ([notebook](ExploratoryDataAnalysis.ipynb), [report](/reports/ExploratoryDataAnalysis.pdf)) the actions taken for Data Cleaning and Feature Engineering are:
* Target normalization
* Handling missing values
* Handling outliers
* Encoding categorical variables
* Scaling continuous variables

And these actions are encapsulated in the method `prepare_data()` from the `utils/data_transformations.py`, but first
the data must be split to create the train and test set to avoid overfitting and inaccurate evaluation.

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=[TARGET], axis=1),data[TARGET], random_state=42, test_size=0.3)

In [10]:
f" Shape x_train {x_train.shape} - y_train {y_train.shape}"

' Shape x_train (14448, 9) - y_train (14448,)'

In [11]:
f" Shape x_test {x_test.shape} - y_test {y_test.shape}"

' Shape x_test (6192, 9) - y_test (6192,)'

# 2. Objectives

This exercise focuses in the predictions of the models to have a better understanding of the features and its impact in the target variable.

# TODO

- Summary of training at least three linear regression models which should be variations that cover using a simple linear regression as a baseline, adding polynomial effects, and using a regularization regression. Preferably, all use the same training and test splits, or the same cross-validation method.
- A paragraph explaining which of your regressions you recommend as a final model that best fits your needs in terms of accuracy and explainability.
- Summary Key Findings and Insights, which walks your reader through the main drivers of your model and insights from your data derived from your linear regression model.
- Suggestions for next steps in analyzing this data, which may include suggesting revisiting this model adding specific data features to achieve a better explanation or a better prediction.

# 3. Linear Regression Models


In [44]:
import numpy
from sklearn.linear_model import LinearRegression, ElasticNetCV, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures

from src.utils.data_transformations import preprocess_data

from sklearn.metrics import mean_squared_error


def rmse(y_true, y_pred):
    return numpy.sqrt(mean_squared_error(y_true, y_pred))

#### Preparing the data

In [13]:
X = x_train.copy()
y = y_train.copy()

X, y, _, _ = preprocess_data(X=X, y=y)
x_test, y_test, _, _ = preprocess_data(X=x_test, y=y_test)

### Simple Linear Regression

In [16]:
lr = GridSearchCV(estimator=LinearRegression(n_jobs=-1),
                  n_jobs=-1,
                  verbose=1,
                  param_grid={},
                  cv=5)

In [17]:
lr.fit(X=X, y=y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=5, estimator=LinearRegression(n_jobs=-1), n_jobs=-1,
             param_grid={}, verbose=1)

In [19]:
r2_score(y_true=y_test, y_pred=lr.predict(x_test))

0.23985303038338823

In [45]:
rmse(y_true=y_test, y_pred=lr.predict(x_test))

99886.10441172331

### Linear Regression with Polynomial Features

In [20]:
model_polyn_pipe = Pipeline(steps=[
    ('polynomail', PolynomialFeatures(degree=3, include_bias=False)),
    ('lr', LinearRegression(n_jobs=-1))
])

lr_polynomial = GridSearchCV(
    estimator=model_polyn_pipe,
    n_jobs=-1,
    verbose=1,
    param_grid={},
    cv=5)

In [21]:
lr_polynomial.fit(X=X, y=y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('polynomail',
                                        PolynomialFeatures(degree=3,
                                                           include_bias=False)),
                                       ('lr', LinearRegression(n_jobs=-1))]),
             n_jobs=-1, param_grid={}, verbose=1)

In [22]:
r2_score(y_true=y_test, y_pred=lr_polynomial.predict(x_test))

-0.1810205852563782

In [46]:
rmse(y_true=y_test, y_pred=lr_polynomial.predict(x_test))

124504.49334473028

### Regression with Regularization

In [40]:
l1_ratios = numpy.linspace(0.1, 0.9, 9)
alphas = numpy.array([1e-5, 5e-5, 0.0001, 0.0005, 0.1, 1, 10])

lr_regularization = ElasticNetCV(
    alphas=alphas,
    l1_ratio=l1_ratios,
    n_jobs=-1,
    max_iter=int(1e4),
    cv=5
)

In [41]:
lr_regularization.fit(X=X, y=y)

ElasticNetCV(alphas=array([1.e-05, 5.e-05, 1.e-04, 5.e-04, 1.e-01, 1.e+00, 1.e+01]),
             cv=5,
             l1_ratio=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
             max_iter=10000, n_jobs=-1)

In [42]:
r2_score(y_true=y_test, y_pred=lr_regularization.predict(x_test))

0.301490231336514

In [47]:
rmse(y_true=y_test, y_pred=lr_regularization.predict(x_test))

95750.82699750476

In [53]:
lr_regularization.alpha_, lr_regularization.l1_ratio_

(5e-05, 0.7000000000000001)

In [48]:
from sklearn.linear_model import LassoCV

alphas2 = numpy.array([1e-5, 5e-5, 0.0001, 0.0005])

lassoCV = LassoCV(alphas=alphas2,
                  max_iter=int(5e4),
                  n_jobs=-1,
                  cv=3)

In [49]:
lassoCV.fit(X=X, y=y)

LassoCV(alphas=array([1.e-05, 5.e-05, 1.e-04, 5.e-04]), cv=3, max_iter=50000,
        n_jobs=-1)

In [50]:
r2_score(y_true=y_test, y_pred=lassoCV.predict(x_test))

0.2398545807056155

In [51]:
rmse(y_true=y_test, y_pred=lassoCV.predict(x_test))

99886.00255265327

In [52]:
lassoCV.alpha_

0.0005

# 4. Insights and key findings


# 5. Next Steps


## Author

<p>Author: Jose Pena</p>
<p>Github: <a href="https://github.com/JoseJuan98">JoseJuan98</a></p>

## Change Log
| Date (YYYY-MM-DD) | Version | Changed By | Change Description   |
|-------------------| ------- | ---------- | -------------------- |
| 2022-11-27        | 1.0     | Jose       | First version        |
